In [14]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
import httpx
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(temperature=0.0,api_key=os.environ['OPENAI_API_KEY'],base_url="https://api-inference.modelscope.cn/v1/", http_client=httpx.Client(verify=False),model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

In [15]:
chat

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001F711C4C230>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001F711C4ECF0>, root_client=<openai.OpenAI object at 0x000001F711C325D0>, root_async_client=<openai.AsyncOpenAI object at 0x000001F711C4C470>, model_name='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api-inference.modelscope.cn/v1/', http_client=<httpx.Client object at 0x000001F70E7C9A90>)

In [16]:
# 美式英语 + 平静、尊敬的语调
style = """American English \
in a calm and respectful tone
"""
text = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [17]:
template_string

'Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'

In [18]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

In [19]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [20]:
customer_messages = prompt_template.format_messages(style=style,text=text)	

In [21]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} response_metadata={}


In [22]:
customer_response = chat(customer_messages)

In [23]:
print(customer_response.content)

Here’s the translation in a style that is American English and calm and respectful:

```
Arrr, I’m fuming that I used my blender lid and splattered my smoothies all over the kitchen walls! And to make it worse, the warranty doesn’t cover the cost of cleaning up my kitchen. I need your help right now, matey!
```
